# Inference of the SEIR model

For background of the purpose of this model, please refer to this [exercise sheet](https://benlambertdotcom.files.wordpress.com/2020/10/week_project.docx).

## Task 1: Code up the ODE and inference model and use these to simulate an epidemic.

In [2]:
import numpy as np

import seirmo


# Define times and parameters
times = np.arange(start=0, stop=40, step=1)  # time in day
initial_values = [0.9, 0, 0.1, 0]
beta = 0.5  # 50% of infected and susceptible entcounters per day lead to transmission
kappa = 0.5  # incubation rate
gamma = 0.05  # recovery rate

# Collect all parameters
parameters = initial_values + [beta] + [kappa] + [gamma]

# Simulate model
model = seirmo.SEIRModel()
result = model.simulate(parameters, times, return_incidence=True)

In [3]:
import plotly.graph_objects as go

# Plot I: Create plot of S, E, I, R
fig = go.Figure()

# Plot of susceptibles
fig.add_trace(
    go.Scatter(
        x=times,
        y=result[:, 0],
        mode='lines',
        name='Susceptible'
    )
)

# Plot of exposed
fig.add_trace(
    go.Scatter(
        x=times,
        y=result[:, 1],
        mode='lines',
        name='Exposed'
    )
)

# Plot of infectious
fig.add_trace(
    go.Scatter(
        x=times,
        y=result[:, 2],
        mode='lines',
        name='Infectious'
    )
)

# Plot of Recovered
fig.add_trace(
    go.Scatter(
        x=times,
        y=result[:, 3],
        mode='lines',
        name='Recovered'
    )
)

# Add axis labels
fig.update_layout(
    xaxis_title='Time in day',
    yaxis_title='Percentage in population'
)

fig.show()

# Plot II: Create plot of incidence number per day
fig = go.Figure()

# Plot of incidences
fig.add_trace(
    go.Bar(
        x=times,
        y=result[:, 4],
        name='Incidences'
    )
)

# Add axis labels
fig.update_layout(
    xaxis_title='Time in day',
    yaxis_title='Incidence number in population percentage'
)

fig.show()

## Task 2: Using either an optimiser or sampling algorithm, estimate the parameters of the model.

### Generate fake data

In [4]:
# Generate fake data
standard_deviation = 0.005
noise = np.random.normal(loc=0, scale=standard_deviation, size=len(times))
incidence_data = result[:, 4] + noise

# Set negative incidences to zero
mask = incidence_data < 0
incidence_data[mask] = np.zeros(shape=len(incidence_data[mask]))

# Visualise data
fig = go.Figure()

# Plot of incidences
fig.add_trace(
    go.Bar(
        x=times,
        y=incidence_data,
        name='Incidences'
    )
)

# Add axis labels
fig.update_layout(
    xaxis_title='Time in day',
    yaxis_title='Incidence number in population percentage'
)

fig.show()

### Infer model parameters

In [5]:
import pints

class SEIRModel(pints.ForwardModel):
    def __init__(self):
        super(SEIRModel, self).__init__()

        self._model = seirmo.SEIRModel()

    def n_parameters(self):
        # Returns number of parameters, i.e. 4 initial conditions and 3 parameters
        return 7

    def n_output(self):
        # Returns number of model outputs.
        # We will only return the incidence number, because this is what we
        # have "measurements" for.
        return 1

    def simulate(self, log_parameters, times):
        output = self._model.simulate(
            parameters=np.exp(log_parameters), times=times, return_incidence=True)
        n_incidence = output[:, 4]

        return n_incidence

In [6]:
# Create starting points of samplers
true_parameters = np.array([0.9, 0, 0.1, 0, 0.5, 0.5, 0.1])

# Shift starting point a little bit just to make it a little bit harder
initial_params = np.log(true_parameters + 0.1)

# Create log-likelihood
pints_model = SEIRModel()
problem = pints.SingleOutputProblem(pints_model, times, incidence_data)
log_likelihood = pints.GaussianKnownSigmaLogLikelihood(problem, sigma=0.005)

# Run sampling routines
optimiser = pints.OptimisationController(
    function=log_likelihood,
    x0=initial_params,
    method=pints.CMAES
)

log_estimates, _ = optimiser.run()
estimates = np.exp(log_estimates)

Maximising LogPDF
Using Covariance Matrix Adaptation Evolution Strategy (CMA-ES)
Running in sequential mode.
Population size: 9
Iter. Eval. Best      Time m:s
0     9     -73.03772   0:00.1
1     18    -37.33264   0:00.1
2     27    -37.33264   0:00.1
3     36    -0.876      0:00.2
20    189    144.2238   0:00.7
40    369    163.9792   0:01.2
60    549    164.7911   0:01.7
80    729    164.8138   0:02.1
100   909    164.8801   0:02.5
120   1089   164.8882   0:02.9
140   1269   164.8891   0:03.4
160   1449   164.8893   0:03.9
180   1629   164.8894   0:04.3
200   1809   164.8894   0:04.8
220   1989   164.8894   0:05.2
240   2169   164.8894   0:05.6
260   2349   164.8894   0:06.1
280   2529   164.8894   0:06.5
300   2709   164.8894   0:06.9
320   2889   164.8894   0:07.3
340   3069   164.8894   0:07.8
360   3249   164.8894   0:08.2
380   3429   164.8894   0:08.6
400   3609   164.8894   0:09.1
420   3789   164.8894   0:09.5
440   3969   164.8894   0:09.9
460   4149   164.8894   0:10.4
480 

In [7]:
print('True parameters:') 
print(true_parameters)
print(' ')
print('Estimates:')
print(estimates)

True parameters:
[0.9 0.  0.1 0.  0.5 0.5 0.1]
 
Estimates:
[0.93632757 0.03016468 0.08094501 0.07024625 0.90142196 0.23193629
 0.11292415]


### Compare inferred model to data

In [8]:
true_incidences = result[:, 4]
inferred_incidences = pints_model.simulate(log_parameters=log_estimates, times=times)

# Plot data and inferred incidences
fig = go.Figure()

# Plot of data
fig.add_trace(
    go.Bar(
        x=times,
        y=true_incidences,
        name='True incidences'
    )
)

# Plot of data
fig.add_trace(
    go.Bar(
        x=times,
        y=inferred_incidences,
        name='Inferred incidences'
    )
)

# Add axis labels
fig.update_layout(
    xaxis_title='Time in day',
    yaxis_title='Incidence number in population percentage'
)

fig.show()

In [17]:
# Get true states
true_s = result[:, 0]
true_e = result[:, 1]
true_i = result[:, 2]
true_r = result[:, 3]
true_incidences = result[:, 4]

# Get inferred states
inferred_result = model.simulate(parameters=estimates, times=times, return_incidence=True)
inferred_s = inferred_result[:, 0]
inferred_e = inferred_result[:, 1]
inferred_i = inferred_result[:, 2]
inferred_r = inferred_result[:, 3]
inferred_incidences = inferred_result[:, 4]

# Plot I: Create plot of true incidence number versus inferred incidence number
fig = go.Figure()

# Plot of true incidences
fig.add_trace(
    go.Bar(
        x=times,
        y=true_incidences,
        name='True incidences'
    )
)

# Plot of inferred incidences
fig.add_trace(
    go.Bar(
        x=times,
        y=inferred_incidences,
        name='Inferred incidences'
    )
)

# Add axis labels
fig.update_layout(
    xaxis_title='Time in day',
    yaxis_title='Incidence number in population percentage'
)

fig.show()

# Plot II: Create plot of true S, E, I, R vs inferred S, E, I, R
fig = go.Figure()

# Plot of true susceptibles
fig.add_trace(
    go.Scatter(
        x=times,
        y=true_s,
        mode='lines',
        name='True S',
        line=dict(color='blue', dash='dash')
    )
)

# Plot of inferred susceptibles
fig.add_trace(
    go.Scatter(
        x=times,
        y=inferred_s,
        mode='lines',
        name='Inferred S',
        line=dict(color='blue')
    )
)

# Plot of true exposed
fig.add_trace(
    go.Scatter(
        x=times,
        y=true_e,
        mode='lines',
        name='True E',
        line=dict(color='red', dash='dash')
    )
)

# Plot of inferred exposed
fig.add_trace(
    go.Scatter(
        x=times,
        y=inferred_e,
        mode='lines',
        name='Inferred E',
        line=dict(color='red')
    )
)

# Plot of true infectious
fig.add_trace(
    go.Scatter(
        x=times,
        y=true_i,
        mode='lines',
        name='True I',
        line=dict(color='green', dash='dash')
    )
)

# Plot of inferred infectious
fig.add_trace(
    go.Scatter(
        x=times,
        y=inferred_i,
        mode='lines',
        name='Inferred I',
        line=dict(color='green')
    )
)

# Plot of true recovered
fig.add_trace(
    go.Scatter(
        x=times,
        y=true_r,
        mode='lines',
        name='True R',
        line=dict(color='black', dash='dash')
    )
)

# Plot of inferred recovered
fig.add_trace(
    go.Scatter(
        x=times,
        y=true_r,
        mode='lines',
        name='Inferred R',
        line=dict(color='black')
    )
)

# Add axis labels
fig.update_layout(
    xaxis_title='Time in day',
    yaxis_title='Percentage in population'
)

fig.show()